In [1]:
%load_ext autoreload
%autoreload 2

import os

import pandas as pd

from kasearch import AlignSequences, PrepareDB, SearchDB, ExtractMetadata

## Prepare query sequence (sequence to search with)

In [2]:
raw_queries = [
    'QVKLQESGAELARPGASVKLSCKASGYTFTNYWMQWVKQRPGQGLDWIGAIYPGDGNTRYTHKFKGKATLTADKSSSTAYMQLSSLASEDSGVYYCARGEGNYAWFAYWGQGTTVTVSS',
    'EVQLQQSGTVLARPGASVKMSCEASGYTFTNYWMHWVKQRPGQGLEWIGAIYPGNSDTSYIQKFKGKAKLTAVTSTTSVYMELSSLTNEDSAVYYCTLYDGYYVFAYWGQGTLVTVSA',
]

In [3]:
query_db = AlignSequences(raw_queries, # Sequences as strings to align.
                          n_jobs=2     # Allocated number for jobs/threads for the search.
                         )
query_db.db.aligned_seqs

array([[81, 86, 75,  0, 76, 81, 69, 83, 71, 65,  0, 69, 76, 65, 82, 80,
        71, 65, 83, 86, 75, 76, 83, 67, 75, 65, 83, 71, 89, 84, 70,  0,
         0,  0,  0,  0,  0,  0,  0,  0, 84, 78, 89, 87, 77, 81,  0, 87,
        86, 75, 81,  0, 82,  0, 80,  0, 71,  0, 81,  0,  0, 71,  0, 76,
        68,  0, 87, 73, 71, 65, 73, 89, 80, 71,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0, 68, 71, 78, 84, 82, 89,  0,  0, 84,  0,  0,
        72,  0,  0, 75, 70,  0,  0, 75,  0,  0,  0, 71, 75, 65, 84, 76,
        84, 65,  0, 68,  0,  0,  0, 75,  0, 83,  0,  0, 83, 83,  0,  0,
         0,  0, 84,  0, 65, 89, 77, 81, 76, 83, 83, 76, 65, 83,  0, 69,
        68, 83, 71, 86, 89, 89, 67, 65, 82, 71, 69, 71, 78,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0, 89, 65, 87, 70, 65, 89, 87, 71,  0, 81,
        71, 84, 84, 86, 84, 86, 83, 83],
       [69, 86, 81,  0, 76, 81, 81, 83, 71, 84,  0, 86, 76, 65, 82, 80,
        71, 65, 83, 86,

--------------
## Initiate search class 

- Default will download a prepared version of OAS to search against
- Alternatively, you can give it the path for a custom database to search against. (See below for how to create a custom database).

In [17]:
DB_PATH = "../data/db-example/"

oasdb = SearchDB(database_path=DB_PATH,       # Path to your database. Default will be to download a prepared version of OAS.
                 allowed_chain='Heavy',       # Search against a specific chain. Default is any chain.
                 allowed_species='Any',       # Search against a specific species. Default is any species.
                 n_jobs=1                 # Allocated number for jobs/threads for the search.
                )

-----------
## Run search

A search against all of OAS takes ~2min for each sequence.


In [18]:
%%timeit

oasdb.search(query_db.db.aligned_seqs[0], # Input can only be a single aligned sequence at a time.
             keep_best_n=10,               # You can define how many most similar sequences to return
             reset_best=True              # In cases where you want to search with the same sequence against multiple databases, you might want to not reset_best. (True is default)
            )             

1.05 s ± 11.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Get N best identities

Identities of the most similar sequence (1st column), CDRs (2nd column) and CDR3 (3rd column) in all of OAS, can be fetched from the object with the bellow command.

In [20]:
oasdb.current_best_identities

array([[0.53370893, 0.5300752 , 0.5833333 ],
       [0.53370893, 0.5275974 , 0.5833333 ],
       [0.51995796, 0.51475155, 0.5833333 ],
       [0.51073253, 0.51428574, 0.5833333 ],
       [0.50833446, 0.51428574, 0.5833333 ],
       [0.50770307, 0.5       , 0.5833333 ],
       [0.5073892 , 0.48590225, 0.5833333 ],
       [0.50597703, 0.47268906, 0.5833333 ],
       [0.50597703, 0.47268906, 0.5608974 ],
       [0.5052413 , 0.4642857 , 0.5608974 ]], dtype=float32)

In [7]:
oasdb.current_best_identities

array([[0.53370893, 0.5300752 , 0.5833333 ],
       [0.53370893, 0.5275974 , 0.5833333 ],
       [0.51995796, 0.51475155, 0.5833333 ],
       [0.50833446, 0.51428574, 0.5833333 ],
       [0.50770307, 0.51428574, 0.5833333 ],
       [0.5073892 , 0.5       , 0.5833333 ],
       [0.50597703, 0.48590225, 0.5833333 ],
       [0.50597703, 0.47268906, 0.5833333 ],
       [0.5052413 , 0.47268906, 0.5       ],
       [0.5046628 , 0.4642857 , 0.5       ]], dtype=float32)

### Get ID's of sequences with the N best identities

ID's of the most similar sequence (1st column), CDRs (2nd column) and CDR3 (3rd column) in all of OAS, can be fetched from the object with the bellow command. 

With ExtractMetadata, these ID's can be used to extract the sequence and its meta data (species, isotype, whole sequence etc.).

In [8]:
oasdb.current_best_ids

array([[[   7, 1705],
        [   9, 1021],
        [   1,    0]],

       [[   6,  365],
        [   9,   74],
        [   1,  229]],

       [[   3,  108],
        [   9,  964],
        [   1,  274]],

       [[   5,  466],
        [   9,  327],
        [   3, 1611]],

       [[   8,   21],
        [   9,  658],
        [   1, 1022]],

       [[   9,  952],
        [   9,  796],
        [   1,  638]],

       [[   7, 1557],
        [   3,  740],
        [   1,  290]],

       [[   5,   82],
        [   3, 2306],
        [   1,  728]],

       [[   9,  200],
        [   3,  108],
        [   3, 2067]],

       [[   9,  151],
        [   9,  243],
        [   6,  528]]], dtype=int32)

---------
## Extract the meta data from matched sequences


The meta data for the N best of the different regions, can be extracted with the following indexes.
- 0: whole sequence
- 1: CDRs
- 2: CDR3

e.g. "oasdb.current_best_ids\[:,0\]\" returns meta data for the N best whole sequence identities.

NB: The column "sequence_alignment_aa" holds the antibody sequence.

In [9]:
meta_db = ExtractMetadata()

In [10]:
n_best_sequences = meta_db.get_meta(oasdb.current_best_ids[:,0])
n_best_sequences

,sequence,locus,stop_codon,vj_in_frame,v_frameshift,productive,rev_comp,complete_vdj,v_call,d_call,...,BType,Vaccine,Disease,Subject,Longitudinal,Organism,Unique sequences,Total sequences,Isotype,Chain
0,GTGGCTGGGAAGGACATACTACAGGTCCAAGTGGTATAATGAATAT...,H,F,T,F,T,F,F,IGHV6-1*01,NaN,...,ASC,None,SARS-COV-2,Patient-1,no,human,3323,6909,IGHM,Heavy
1,TGGGGTTTTGATCCTGAAGATGGTGAAACAATATACGCACAGAAGT...,H,F,T,F,T,T,F,IGHV1-24*01,IGHD3-10*01,...,ASC,None,SARS-COV-2,Patient-1,no,human,1184,1602,IGHG,Heavy
2,CTAGTGATCTTGCCGTGCGTCGAGGCTCTCTGATATCGGGATTCAC...,H,F,T,F,T,F,F,IGHV3-48*03,IGHD3-10*01,...,Naive-B-Cells,None,SARS-COV-2,Patient-1,no,human,2423,8445,IGHM,Heavy
3,TTCACCATCTCCAGAGACAATTCCAAGAACACGCTGTATCTGCAAA...,H,F,T,F,T,T,F,IGHV3-23*01,IGHD1-1*01,...,Naive-B-Cells,None,SARS-COV-2,Patient-1,no,human,491,1234,IGHD,Heavy
4,GGGTGCGACAGGCCCCTGGACAAGGGCTTGAGTGGATGGGAGGGAT...,H,F,T,F,T,T,F,IGHV1-69*01,IGHD3-10*01,...,Naive-B-Cells,None,SARS-COV-2,Patient-1,no,human,33,84,IGHG,Heavy
5,GCCTCTATCGCGATCGGCAGAGCTCATATCTTATATCCTGGAAGCA...,H,F,T,F,T,F,F,IGHV4-34*01,IGHD3-10*01,...,Naive-B-Cells,None,SARS-COV-2,Patient-1,no,human,2619,4005,Bulk,Heavy
6,AAATGCCTCATCGCTCGGCTCCGGGCTATCTTAGCTGCGAAGCAAA...,H,F,T,F,T,T,F,IGHV3-15*01,IGHD3-22*01,...,ASC,None,SARS-COV-2,Patient-1,no,human,3323,6909,IGHM,Heavy
7,ACAATCTACGCACAGAAGTTCCAGGGCAGAGTCACCATGACCGAGG...,H,F,T,F,T,T,F,IGHV1-24*01,NaN,...,Naive-B-Cells,None,SARS-COV-2,Patient-1,no,human,491,1234,IGHD,Heavy
8,CTCGGGAGTTGGAGGTTCGAATCGTCTTTCTTATATCTGGGGGCTG...,H,F,T,F,T,F,F,IGHV4-38-2*01,IGHD2-15*01,...,Naive-B-Cells,None,SARS-COV-2,Patient-1,no,human,2619,4005,Bulk,Heavy
9,TGGTTCCAGGACGAAAACCATCACCGATTCTTATATCCGGGAGCTA...,H,F,T,F,T,F,F,IGHV3-30*18,IGHD3-10*01,...,Naive-B-Cells,None,SARS-COV-2,Patient-1,no,human,2619,4005,Bulk,Heavy


In [11]:
n_best_sequences.sequence_alignment_aa.values

array(['WLGRTYYRSKWYNEYAVSVKGRITINPDTSKNQFSLQLNSVTPEDTAVYYCSRARDGYVDFWGQGTLVTVSS',
       'GFDPEDGETIYAQKFQGRVTMTEDTSTDTAYMELSSLRSEDTAVYYCATASPITIHHNWFDPWGQGTLVTVSS',
       'GSLISGFTISRDNAKNSLYLQMNSLRAEDTAVYYCASLWFGELSRLYYFDYWGQGTLVTVSS',
       'FTISRDNSKNTLYLQMNSLRAEDTAVYYCAANWPPGDYYYMDVWGKGTTVTVSS',
       'VRQAPGQGLEWMGGIIPIFGTANYAQKFQGRVTITADESTSTAYMELSSLRSEDTAVYYCARDGGFGDYYMDVWGKGTTVTVSS',
       'GSTNYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCARGSELTMVRGMDVWGKGTTVTVSS',
       'SKTDGGTADYAAPVKGRFTISRDDAKNTLYLQMSSLKTEDIAVYYCTTDPDDSSGGWGQGTLVTVAS',
       'TIYAQKFQGRVTMTEDTSTDTAYMELSSLRSEDPAVYYCATAPPNFDYWGQGTLVTVSS',
       'GWIRQPPGKGLEWIGSIYHSGSTYYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCARWLGYCSGGSCYSGLRDAFD',
       'SYGMHWVRQAPGKGLEWVAVISYDGSNKYYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAKDSKLYYYGSGDPYYM'],
      dtype=object)

----------
## Create custom database


To create your own database you first need to create a csv file in the OAS format. For an example file, look at data/custom-data-example.csv. This file consists of a dictionary containing the metadata in the first line and then rows of the individual sequences afterwards. Only the Species and Chain is strictly needed in the metadata, and only the amino acids sequence of the antibodies is required for each antibody sequence.

### 1. Format your data into OAS files

In [12]:
import json

from kasearch.species_anarci import number
from kasearch.merge_db import merge_files

In [13]:
metadata = {"Species":"Human", "Chain":"Heavy"}
metadata = pd.Series(name=json.dumps(metadata), dtype='object')
seqsdata = pd.DataFrame([["EVQLVESGGGLAKPGGSLRLHCAASGFAFSSYWMNWVRQAPGKRLEWVSAINLGGGLTYYAASVKGRFTISRDNSKNTLSLQMNSLRAEDTAVYYCATDYCSSTYCSPVGDYWGQGVLVTVSS"],
                          ["EVQLVQSGAEVKRPGESLKISCKTSGYSFTSYWISWVRQMPGKGLEWMGAIDPSDSDTRYNPSFQGQVTISADKSISTAYLQWSRLKASDTATYYCAIKKYCTGSGCRRWYFDLWGPGT"]
                         ], columns = ['sequence_alignment_aa'])

In [14]:
save_file = "../data/custom-data-example-2.csv"
metadata.to_csv(save_file, index=False)
seqsdata.to_csv(save_file, index=False, mode='a')

### 2. Turn your OAS formatted files into a custom database

After creating all the files you want to include in the new database, you can run the following code to create the database.

In [15]:
db_folder = "../data/my_db"
db_files = ['../data/custom-data-example.csv']

In [16]:
newDB = PrepareDB(db_path=db_folder)

db_dict = {}
for num, data_unit_file in enumerate(db_files):

    db_dict[num] = data_unit_file
    
    metadata = json.loads(','.join(pd.read_csv(data_unit_file, nrows=0).columns))
    seqsdata = pd.read_csv(data_unit_file, header=1, usecols=['sequence_alignment_aa']).iloc[:,0].values
    numbered_sequences = [number(sequence, allowed_species=None)[0] for sequence in seqsdata]

    newDB.prepare_database(numbered_sequences, 
                           file_id=num, 
                           chain=metadata['Chain'], 
                           species=metadata['Species'])
    
newDB.save_database()

merge_files(db_folder)

with open(os.path.join(db_folder, "my_db_id_to_study.txt"), "w") as handle: handle.write(str(db_dict))

KeyboardInterrupt: 

###You can now create your own database to search from.

In [ ]:
mydb = SearchDB(database_path=db_folder,      # Path to your database. Default will be to download a prepared version of OAS.
                 allowed_chain='Heavy',       # Search against a specific chain. Default is any chain.
                 allowed_species='Any',       # Search against a specific species. Default is any species.
                 n_jobs=1                     # Allocated number for jobs/threads for the search.
                )

In [ ]:
mydb.search(query_db.db.aligned_seqs[0])

IndexError: list index out of range

In [ ]:
mydb.current_best_identities

array([[0.64705884, 0.60714287, 0.5833333 ],
       [0.64705884, 0.60714287, 0.5833333 ],
       [0.64705884, 0.60714287, 0.5833333 ],
       [0.64136165, 0.577765  , 0.56666666],
       [0.64136165, 0.577765  , 0.56666666],
       [0.64136165, 0.577765  , 0.56666666],
       [0.6315294 , 0.57424814, 0.56666666],
       [0.6315294 , 0.57424814, 0.56666666],
       [0.6315294 , 0.57424814, 0.56666666],
       [0.63080317, 0.57424814, 0.55      ]], dtype=float32)